# Previsão de Votação de Deputados

## A tarefa

Nesta atividade construiremos modelos preditivos de regressão em scikit learn para a predição dos votos de deputados federais considerando as últimas eleições. As atividades esperadas para essa etapa são descritas a seguir:

1. Baixe os dados [aqui](https://canvas.instructure.com/courses/1389733/files/69523670/download?verifier=A5EPvssqIQCjlxpWQyesLFer1VDTlRyTAAXR2iyi&wrap=1).
2. Considere o pipeline mostrado nesse [link](https://www.kaggle.com/apapiu/regularized-linear-models) para construir seus modelos de regressão. Isso implica, dentre outras coisas:
    1. Analisar as distribuições das variáveis para ver se estão enviesadas e precisam de correção; tratamento de valores ausentes, variáveis categóricas e normalização, quando for o caso.
    2. Construir modelos de regressão com (ridge e lasso) e sem regularização.
    3. Considerar também modelos de regressão não paramétrica como K-NN.
    4. Considerar outros modelos ainda não vistos em sala de sua escolha (e.g. SVR, Regression Trees e Random Florests).
    5. Tunar os hiperâmetros para cada caso e retornar os rmses de validação cruzada para todos os modelos avaliados.
    6. Plotar os resíduos versus predições e analisar se esses plots representam bons indícios da adequabilidade dos modelos a esse problema.
3. Alguns dias antes da entrega final serão liberados os dados de teste referentes à 2014 para validação final dos seus melhores modelos.
    1. Dica: Uma coisa que você pode fazer é usar os dados de 2006 como treino e os de 2010 como validação. Uma vez encontrados os melhores modelos para 2010 junte 2006+2010, retreine, e aplique o modelo aos dados de 2014 que serão liberados.
4. Responder:
    1. Dentre os modelos avaliados, qual foi o que deu o melhor resultado nos dados de 2014 em termos de RMSE? Justifique bem sua resposta.
A entrega deve ser um notebook Jupyter com código python e texto explicativo quando necessário. Crie um repositório na sua conta do github e envie o link do html do notebook.

## Sobre os dados

Vamos explorar dados sobre as votações que candidatos à Câmara Federal de Deputados receberam nos anos de 2006 e 2010. Esses dados foram extraídos do [TSE](http://www.tse.jus.br/hotSites/pesquisas-eleitorais/index.html), pré-processados e contemplam informações sobre aproximadamente 7.300 candidatos. A descrição de cada atributo é dada mais abaixo.

|Variável                                   | Tipo | Descrição |
|-------------------------------------------|------|-----------|
|"sequencial_candidato"| (character) | id do candidato|
|"nome"| (character) | |
|"uf" | (character) | |
|"partido" | (character)| |
|"quantidade_doacoes"| (integer)| |
|"quantidade_doadores"| (integer) | número de doadores diferentes|
|"total_receita" | (double) | soma em R\$ das doações |
|"media_receita" | (double) | média das doações |
|"recursos_de_outros_candidatos/comites" | (double) | quantia em R\$ das doações provenientes de outros candidatos ou comite partidário |
|"recursos_de_pessoas_fisicas" | (double) | quantia em R\$ das doações provenientes de outros CPFs |
|"recursos_de_pessoas_juridicas" | (double) | quantia em R\$ das doações provenientes de outros CNPJ |
|"recursos_proprios" | (double) | quantia em R\$ das doações provenientes do próprio candidato |
|"recursos_de_partido_politico" | (double) | quantia em R\$ das doações provenientes do partido político do candidato |
|**"votos"** | (integer) | **variável alvo**. Se refere ao número de votos na campanha de 2006 e 2010 |
|"quantidade_despesas" | (integer) | |
|"quantidade_fornecedores" | (integer) | número de fornecedores/despesas diferentes |
|"total_despesa" | (double) | soma em R$ das despesas de campanha |
|"media_despesa" | (double) | média das despesas de campanha |
|"cargo" |(character) | |
|"Sexo": | (character) | |
|"grau": | (character) | grau de instrução do candidato |
|"estado_civil" | (character) |
|"ocupacao" | (character) | ocupação do candidato |

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import imblearn

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [2]:
# Para regressão linear sem regularização
from sklearn.linear_model import LinearRegression
# Para regressão linear com regularização
from sklearn import metrics
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, Lasso, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score
# não paramétrica K-NN
from sklearn.neighbors import KNeighborsRegressor
# random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_regression

In [126]:
# Carregando
eleicoes_train = pd.read_csv('../data/train.csv')
eleicoes_test = pd.read_csv('../data/test.csv')
directions = pd.read_csv('../data/directions.csv')
regions = pd.read_csv('../data/regions.csv')

In [127]:

direction2=[]
region2=[]
for index,row in eleicoes_train.iterrows():
    direction2.append(directions[row['partido']][0])  
    region2.append(regions[row['uf']][0])    
eleicoes_train['direction'] = direction2  
eleicoes_train['region'] = region2
direction2=[]
region2=[]
for index,row in eleicoes_test.iterrows():
    direction2.append(directions[row['partido']][0])
    region2.append(regions[row['uf']][0])    
eleicoes_test['direction'] = direction2
eleicoes_test['region'] = region2


In [128]:
eleicoes_test.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,direction,region
0,2014,10000000135,EMERSON DA SILVA SANTOS,AC,PSOL,3,3,1580.00,526.67,0.00,...,3,1580.00,526.67,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),"CORRETOR DE IMÓVEIS, SEGUROS, TÍTULOS E VALORES",L,NO
1,2014,10000000142,GERALDO SILVA DOS SANTOS,AC,PSOL,5,5,3180.00,636.00,0.00,...,5,3130.02,521.67,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),VIGILANTE,L,NO
2,2014,10000000158,CARLOS CESAR CORREIA DE MESSIAS,AC,PSB,40,38,336793.13,8419.83,1923.07,...,139,326869.78,2254.27,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,CASADO(A),OUTROS,L,NO
3,2014,10000000161,IDESIO LUIS FRANKE,AC,PT,29,29,156719.32,5404.11,39122.32,...,121,241016.07,1772.18,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),AGRÔNOMO,L,NO
4,2014,10000000163,LEONARDO CUNHA DE BRITO,AC,PT,160,146,737073.00,4606.71,10000.00,...,354,567401.15,1095.37,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),ADVOGADO,L,NO


In [129]:




#eleicoes = pd.concat((eleicoes_train,eleicoes_test),sort='false')
eleicoes = eleicoes_train
eleicoes = eleicoes.reindex(columns=eleicoes.columns)
eleicoes_test = eleicoes_test.reindex(columns=eleicoes_test.columns)


eleicoes, eleicoes_val = train_test_split(eleicoes, test_size=0.3, random_state=0)

eleicoes, eleicoes_val2 = train_test_split(eleicoes, test_size=0.427, random_state=0)

eleicoes = eleicoes.fillna(eleicoes.mean())
eleicoes_val = eleicoes_val.fillna(eleicoes_val.mean())
eleicoes_val2 = eleicoes_val2.fillna(eleicoes_val2.mean())



In [ ]:
eleicoes_train.head()

In [ ]:
eleicoes_test.head()

In [ ]:
eleicoes_val.head()

In [ ]:
with sns.axes_style(style='ticks'):
    g = sns.factorplot("situacao", "media_despesa", "ano", data=eleicoes, kind="box")
    g.set_axis_labels("Situação", "Despesas médias");

In [4]:
eleitos = eleicoes.loc[:,[
       'ano', 'situacao'
    ]]

eleitos_test = eleicoes_test.loc[:,[
       'ano', 'situacao'
    ]]

eleitos_val = eleicoes_val.loc[:,[
       'ano', 'situacao'
    ]]

eleitos_val2 = eleicoes_val2.loc[:,[
       'ano', 'situacao'
    ]]


#eleitos = eleitos.reindex(columns=eleitos.columns)
#eleitos_test = eleitos_test.reindex(columns=eleitos_test.columns)
#eleitos_val = eleitos_val.reindex(columns=eleitos_val.columns)
#eleitos_val2 = eleitos_val2.reindex(columns=eleitos_val2.columns)

eleitos = eleitos.fillna(eleitos.mean())
eleitos_test = eleitos_test.fillna(eleitos_test.mean())
eleitos_val = eleitos_val.fillna(eleitos_val.mean())
eleitos_val2 = eleitos_val2.fillna(eleitos_val2.mean())

C:\Users\tclem\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [130]:


eleicoes = eleicoes.loc[:,[
       'uf', 'ano', 'situacao', 'sequencial_candidato', 'nome', 'partido', 'quantidade_doacoes', 'total_receita', 'quantidade_doadores', 
       'recursos_de_outros_candidatos.comites', 'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
       'recursos_proprios', 'recursos_de_partido_politico', 'quantidade_despesas', 'quantidade_fornecedores',
       'total_despesa', 'cargo', 'sexo', 'grau_instrucao',
       'estado_civil', 'ocupacao', 'direction', 'region'
    ]]
eleicoes_test = eleicoes_test.loc[:,[
       'uf', 'ano', 'situacao', 'sequencial_candidato', 'nome', 'partido', 'quantidade_doacoes', 'total_receita', 'quantidade_doadores', 
       'recursos_de_outros_candidatos.comites', 'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
       'recursos_proprios', 'recursos_de_partido_politico', 'quantidade_despesas', 'quantidade_fornecedores',
       'total_despesa', 'cargo', 'sexo', 'grau_instrucao',
       'estado_civil', 'ocupacao', 'direction', 'region'
    ]]
eleicoes_val = eleicoes_val.loc[:,[
       'uf', 'ano', 'situacao', 'sequencial_candidato', 'nome', 'partido', 'quantidade_doacoes', 'total_receita', 'quantidade_doadores', 
       'recursos_de_outros_candidatos.comites', 'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
       'recursos_proprios', 'recursos_de_partido_politico', 'quantidade_despesas', 'quantidade_fornecedores',
       'total_despesa', 'cargo', 'sexo', 'grau_instrucao',
       'estado_civil', 'ocupacao', 'direction', 'region'
    ]]
eleicoes_val2 = eleicoes_val2.loc[:,[
       'uf', 'ano', 'situacao', 'sequencial_candidato', 'nome', 'partido', 'quantidade_doacoes', 'total_receita', 'quantidade_doadores', 
       'recursos_de_outros_candidatos.comites', 'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
       'recursos_proprios', 'recursos_de_partido_politico', 'quantidade_despesas', 'quantidade_fornecedores',
       'total_despesa', 'cargo', 'sexo', 'grau_instrucao',
       'estado_civil', 'ocupacao', 'direction', 'region'
    ]]

C:\Users\tclem\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [131]:
eleicoes = pd.get_dummies(eleicoes, 
                          prefix=['region', 'direction', 'ocupacao','situacao'], 
                          columns=['region', 'direction', 'ocupacao','situacao'])
eleicoes_test = pd.get_dummies(eleicoes_test, 
                          prefix=['region', 'direction', 'ocupacao','situacao'], 
                          columns=['region', 'direction', 'ocupacao','situacao'])
eleicoes_val = pd.get_dummies(eleicoes_val, 
                          prefix=['region', 'direction', 'ocupacao','situacao'], 
                          columns=['region', 'direction', 'ocupacao','situacao'])
eleicoes_val2 = pd.get_dummies(eleicoes_val2, 
                          prefix=['region', 'direction', 'ocupacao','situacao'], 
                          columns=['region', 'direction', 'ocupacao','situacao'])


In [ ]:

    
    
with sns.axes_style('white'):
    g = sns.factorplot("situacao", data=eleitos, aspect=2,
                       kind="count", color='steelblue')
    g.set_xticklabels(step=5)

In [ ]:




# Pré-Processamento inicial para evitar número de colunas diferente entre as bases de tesate e treino
eleicoes = pd.get_dummies(eleicoes, 
                          prefix=['uf', 'partido', 'cargo', 'sexo', 'grau_instrucao', 'estado_civil', 'ocupacao'], 
                          columns=['uf', 'partido', 'cargo', 'sexo', 'grau_instrucao', 'estado_civil', 'ocupacao'])

#eleicoes['recursos'] = (eleicoes.test_one + df.test_two)

#features = eleicoes.loc[eleicoes['ano'] != 2014].drop(columns=['votos'])
features = eleicoes.loc[eleicoes['ano'] != 2014]
#target = eleicoes.loc[eleicoes['ano'] != 2014].votos
target = eleitos.loc[eleitos['ano'] != 2014].drop(columns=['ano'])

#features = features.loc[:,[
#       'uf', 'quantidade_doacoes', 'total_receita', 'partido', 'quantidade_doadores', 
#       'recursos_de_outros_candidatos/comites',
#       'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
#       'recursos_proprios', 'quantidade_despesas', 'quantidade_fornecedores',
#       'total_despesa', 'cargo', 'sexo', 'grau',
#       'estado_civil', 'ocupacao'
#    ]]

x_test_final = eleicoes.loc[eleicoes['ano'] == 2014]
y_test_final = eleitos.loc[eleitos['ano'] == 2014].drop(columns=['ano'])

#x_teste_final = x_teste_final.loc[:,[
#       'uf', 'quantidade_doacoes', 'total_receita', 'partido', 'quantidade_doadores', 
#       'recursos_de_outros_candidatos/comites',
#       'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
#       'recursos_proprios', 'quantidade_despesas', 'quantidade_fornecedores',
#       'total_despesa', 'cargo', 'sexo', 'grau',
#       'estado_civil', 'ocupacao'
#    ]]

#treino 75% e validação 25%
x_treino, x_validacao, y_treino, y_validacao = train_test_split(features, target, random_state = 8)

x_treino_final = features
y_treino_final = target

In [ ]:
eleicoes.head()

In [ ]:
#def plot_2d_space(X, y, label='Classes'):   
#    colors = ['#770000', '#1FC7E4']
#    markers = ['o', 's']
#    for l, c, m in zip(np.unique(y), colors, markers):
#        plt.scatter(
#            X[y==l, 0],
#            X[y==l, 1],
#            c=c, label=l, marker=m, alpha =0.1
#        )
#    plt.title(label)
#    plt.legend(loc='upper right')
#    plt.show()

def plot_2d_space(X, y, label='Classes'):   
    colors = ['#1F77B4', '#FF7F0E']
    markers = ['o', 's']
    for l, c, m in zip(np.unique(y), colors, markers):
        plt.scatter(
            X[y==l, 0],
            X[y==l, 1],
            c=c, label=l, marker=m, alpha =0.1
        )
    plt.title(label)
    plt.legend(loc='upper right')
    plt.show()

def plot_2d_space_logy(X, y, label='Classes'):   
    colors = ['purple', 'yellow']
    markers = ['o', 's']
    for l, c, m in zip(np.unique(y), colors, markers):
        plt.scatter(
            X[y==l, 0],
            np.log1p(X[y==l, 1]),
            c=c, label=l, marker=m, alpha =0.5
        )
    plt.title(label)
    plt.legend(loc='upper right')
    plt.show()
    
#from sklearn.decomposition import PCA

#non_cols = ['sequencial_candidato', 'nome', 'uf', 'partido', 'cargo', 'sexo', 'grau_instrucao', 'estado_civil', 'ocupacao'] 
# Takes all 47 other columns
#cols = list(set(eleicoes.columns) - set(non_cols))

#pca = PCA(n_components=2)
#X = pca.fit_transform(eleicoes.loc[:,cols])

#plot_2d_space(X, pd.get_dummies(eleitos.situacao).nao_eleito , 'Imbalanced dataset (2 PCA components)')





from imblearn.under_sampling import RandomUnderSampler

non_cols = ['sequencial_candidato', 'nome'] 


cols = ['total_receita', 'quantidade_doadores', 'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
       'recursos_proprios']

colors = ['#770000', '#1FC7E4']
markers = ['o', 's']

c = pd.get_dummies(eleitos.situacao).nao_eleito
l = np.unique(c)

plt.scatter(
    eleicoes["total_receita"],
    np.log1p(eleicoes["quantidade_doadores"]),
    c=c, alpha =0.5
)

plt.legend(loc='upper right')
plt.show()
#cols = list(set(eleicoes.columns) - set(non_cols))

rus = RandomUnderSampler(return_indices=True)
X_rus, y_rus, id_rus = rus.fit_sample(eleicoes.loc[:,cols], pd.get_dummies(eleitos.situacao).nao_eleito)

print('Removed indexes:', id_rus)

#plot_2d_space(X_rus, y_rus, 'Random under-sampling')

#plt.hist(y_rus, density =1)

df = pd.DataFrame(X_rus)
df['situacao'] = y_rus
df.situacao.value_counts().plot(kind='bar', title='Under 1 Count (situacao)');



#plt.hist(y_rus, 3, facecolor='green', alpha=0.75)

#plt.xlabel('Smarts')
#plt.ylabel('Probability')
#plt.title(r'$\mathrm{Histogram\ of\ IQ:}\ \mu=100,\ \sigma=15$')
#plt.axis([0,1,0,1100])
#plt.grid(True)

#plt.show()


#eleitos.situacao.value_counts().plot(kind='bar', title='Count (target)');





In [ ]:
# Class count
count_class_0, count_class_1 = eleicoes_train.situacao.value_counts()

# Divide by class
df_class_0 = eleicoes_train[eleicoes_train['situacao'] == 'nao_eleito']
df_class_1 = eleicoes_train[eleicoes_train['situacao'] == 'eleito']



df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.situacao.value_counts())

df_test_under.situacao.value_counts().plot(kind='bar', title='Under Count (situacao)');

In [ ]:


df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.situacao.value_counts())

df_test_over.situacao.value_counts().plot(kind='bar', title='Over Count (situacao)');

In [ ]:
#Under-sampling: Tomek links

from imblearn.under_sampling import TomekLinks

tl = TomekLinks(return_indices=True, ratio='majority')
X_tl, y_tl, id_tl = tl.fit_sample(eleicoes.loc[:,["total_receita","quantidade_doadores"]], pd.get_dummies(eleitos.situacao).nao_eleito)

print('Removed indexes:', id_tl)




plot_2d_space_logy(X_tl, y_tl, 'Tomek links under-sampling')

plt.scatter(
    eleicoes["total_receita"],
    np.log1p(eleicoes["quantidade_doadores"]),
    c=pd.get_dummies(eleitos.situacao).nao_eleito, alpha =0.5
)
plt.legend(loc='upper right')
plt.show()

df = pd.DataFrame(X_tl)
df['situacao'] = y_tl
df.situacao.value_counts().plot(kind='bar', title='Tomek Links Count (situacao)');

In [ ]:
#Cluster Centroids
from imblearn.under_sampling import ClusterCentroids

#cc = ClusterCentroids()
cc = ClusterCentroids(ratio={1:1000})
X_cc, y_cc = cc.fit_sample(eleicoes.loc[:,["total_receita","quantidade_doadores"]], pd.get_dummies(eleitos.situacao).nao_eleito)

plot_2d_space_logy(X_cc, y_cc, 'Cluster Centroids under-sampling')

plt.scatter(
    eleicoes["total_receita"],
    np.log1p(eleicoes["quantidade_doadores"]),
    c=pd.get_dummies(eleitos.situacao).nao_eleito, alpha =0.5
)
plt.legend(loc='upper right')
plt.show()

df = pd.DataFrame(X_cc)
df['situacao'] = y_cc
df.situacao.value_counts().plot(kind='bar', title='Cluster Centroids Count (situacao)');

In [ ]:

#SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(eleicoes_train.loc[:,["total_receita","quantidade_doadores"]], pd.get_dummies(eleitos.situacao).nao_eleito)

plot_2d_space_logy(X_sm, y_sm, 'SMOTE over-sampling')

plt.scatter(
    eleicoes["total_receita"],
    np.log1p(eleicoes["quantidade_doadores"]),
    c=pd.get_dummies(eleitos.situacao).nao_eleito, alpha =0.5
)
plt.legend(loc='upper right')
plt.show()


plt.scatter(
    eleicoes["recursos_de_outros_candidatos.comites"],
    np.log1p(eleicoes["recursos_proprios"]),
    c=pd.get_dummies(eleitos.situacao).nao_eleito, alpha =0.5
)
plt.legend(loc='upper right')
plt.show()





plt.scatter(
    eleicoes["recursos_de_partido_politico"],
    eleicoes["recursos_de_pessoas_juridicas"],
    c=pd.get_dummies(eleitos.situacao).nao_eleito, alpha =0.5
)
plt.legend(loc='upper right')
plt.show()

df = pd.DataFrame(X_sm)
df['situacao'] = y_sm
df.situacao.value_counts().plot(kind='bar', title='SMOTE Count (situacao)');

In [ ]:
#sns.set_style('darkgrid')
#sns.distplot(np.log1p(eleicoes["recursos_proprios"]))


orderer = eleitos.situacao.value_counts().index
eleicoes2=eleicoes
eleicoes2['situacao'] = eleitos.situacao
eleicoes2["recursos_proprios"]=np.log1p(eleicoes2["recursos_proprios"])
g = sns.FacetGrid(eleicoes2, row="situacao", row_order=orderer,
                  aspect=4)
g.map(sns.distplot, "recursos_proprios", hist=True, rug=True);

In [ ]:
orderer = eleitos.situacao.value_counts().index
#eleicoes2=eleicoes
eleicoes2["recursos_de_outros_candidatos.comites"]=np.log1p(eleicoes2["recursos_de_outros_candidatos.comites"])
g = sns.FacetGrid(eleicoes2, row="situacao", row_order=orderer,
                  aspect=4)
g.map(sns.distplot, "recursos_de_outros_candidatos.comites", hist=True, rug=True);

In [ ]:
orderer = eleitos.situacao.value_counts().index
#eleicoes2=eleicoes
eleicoes2["recursos_de_partido_politico"]=eleicoes2["recursos_de_partido_politico"]
g = sns.FacetGrid(eleicoes2, row="situacao", row_order=orderer,
                  aspect=4)
g.map(sns.distplot, "recursos_de_partido_politico", hist=True, rug=True);

In [ ]:


orderer = eleitos.situacao.value_counts().index
#eleicoes2=eleicoes
eleicoes2["recursos_de_pessoas_juridicas"]=np.log1p(eleicoes2["recursos_de_pessoas_juridicas"])
g = sns.FacetGrid(eleicoes2, row="situacao", row_order=orderer,
                  aspect=4)
g.map(sns.distplot, "recursos_de_pessoas_juridicas", hist=True, rug=True);

In [ ]:

#orderer = eleicoes2.partido.value_counts().index

#eleicoes2["recursos_de_pessoas_fisicas"]=np.log1p(eleicoes2["recursos_de_pessoas_fisicas"])
#g = sns.FacetGrid(eleicoes2.loc[eleicoes['situacao'] == 'eleito'], row="partido", row_order=orderer,
#                  aspect=4)
#g.map(sns.distplot, "recursos_de_pessoas_fisicas", hist=True, rug=True);

orderer = eleitos.situacao.value_counts().index
#eleicoes2=eleicoes
eleicoes2["recursos_de_pessoas_fisicas"]=np.log1p(eleicoes2["recursos_de_pessoas_fisicas"])
g = sns.FacetGrid(eleicoes2, row="situacao", row_order=orderer,
                  aspect=4)
g.map(sns.distplot, "recursos_de_pessoas_fisicas", hist=True, rug=True);

In [ ]:

orderer = eleitos.situacao.value_counts().index
#eleicoes2=eleicoes
eleicoes2["quantidade_doadores"]=np.log1p(eleicoes2["quantidade_doadores"])
g = sns.FacetGrid(eleicoes2, row="situacao", row_order=orderer,
                  aspect=4)
g.map(sns.distplot, "quantidade_doadores", hist=True, rug=True);

In [273]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(ratio='auto')
X_smt, y_smt = smt.fit_sample(eleicoes.loc[:,['ocupacao_DEPUTADO','recursos_proprios','recursos_de_pessoas_fisicas','recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']], eleicoes.eleito)

plot_2d_space_logy(X_smt, y_smt, 'SMOTE + Tomek links')

plt.scatter(
    eleicoes["total_receita"],
    np.log1p(eleicoes["quantidade_doadores"]),
    c=pd.get_dummies(eleitos.situacao).nao_eleito, alpha =0.5
)
plt.legend(loc='upper right')
plt.show()

df = pd.DataFrame(X_smt)
df['situacao'] = y_smt
df.situacao.value_counts().plot(kind='bar', title='SMOTE + Tomek links Count (situacao)');

AttributeError: 'DataFrame' object has no attribute 'eleito'

In [ ]:
from sklearn.model_selection import StratifiedKFold
#import math
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    #test_pred=np.empty((test.shape[0],1),float)
    test_pred=None
    train_pred=np.empty((0,1),float)
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        #test_pred=np.append(test_pred,model.predict(test))
        if test_pred is None:
            test_pred=pd.DataFrame(model.predict(test))
        else:
            test_pred=pd.concat([test_pred, pd.DataFrame(model.predict(test))], axis=1)
    #return test_pred.reshape(-1,1),train_pred
    return test_pred,train_pred


In [265]:
def rmse_cv(model,x,y):#x é dataframe e y é list
    #rmse= np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = 5, scoring='recall'))
    rmse= np.sqrt(-cross_val_score(model, x, y, cv = 5, scoring='precision'))
    return(rmse)

#def rmse_cv_final(model):
#    rmse= np.sqrt(-cross_val_score(model, x_teste_final, y_teste_final.votos, scoring="neg_mean_squared_error", cv = 5))
#    return(rmse)

In [56]:
np.unique(eleicoes_val.loc[:,['uf']])

array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
       'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
       'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object)

In [13]:
eleicoes = pd.get_dummies(eleicoes, 
                          prefix=['situacao'], 
                          columns=['situacao'])

eleicoes_val = pd.get_dummies(eleicoes_val, 
                          prefix=['situacao'], 
                          columns=['situacao'])

eleicoes_val2 = pd.get_dummies(eleicoes_val2, 
                          prefix=['situacao'], 
                          columns=['situacao'])


In [133]:

eleicoes = eleicoes.reset_index()
eleicoes_val = eleicoes_val.reset_index()
eleicoes_val2 = eleicoes_val2.reset_index()

In [266]:
from sklearn import tree

model1 = tree.DecisionTreeClassifier(criterion = "entropy", splitter = 'random', max_leaf_nodes = 12, min_samples_leaf = 15, max_depth= 10)
#model1.fit(eleicoes.loc[:,['recursos_de_pessoas_juridicas']], eleicoes.loc[:,['situacao_eleito']])
model1.fit(eleicoes.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']], eleicoes.loc[:,['situacao_eleito']])
#val_pred1=model1.predict(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas']])
val_pred1=model1.predict(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']])
#test_pred1=model1.predict(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas']])
test_pred1=model1.predict(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']])
val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)

print(rmse_cv(model1,eleicoes.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes.loc[:,['situacao_eleito']]))

print(model1.score(eleicoes.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes.loc[:,['situacao_eleito']]))
print(model1.score(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes_val.loc[:,['situacao_eleito']]))
print(model1.score(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes_val2.loc[:,['situacao_eleito']]))

[nan nan nan nan nan]
0.8825261780104712
0.8762571053782248
0.8911803422553751


In [261]:
from sklearn.ensemble import GradientBoostingClassifier

model1 = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
#model1.fit(eleicoes.loc[:,['recursos_de_pessoas_juridicas']], eleicoes.loc[:,['situacao_eleito']])
model1.fit(eleicoes.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']], eleicoes.loc[:,['situacao_eleito']])
#val_pred1=model1.predict(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas']])
val_pred1=model1.predict(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']])
#test_pred1=model1.predict(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas']])
test_pred1=model1.predict(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']])
val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)

print(rmse_cv(model1,eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes_val2.loc[:,['situacao_eleito']]))

print(model1.score(eleicoes.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes.loc[:,['situacao_eleito']]))
print(model1.score(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes_val.loc[:,['situacao_eleito']]))
print(model1.score(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes_val2.loc[:,['situacao_eleito']]))

C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[0.31726401 0.320694   0.29277002 0.31448545 0.32139804]
0.8992146596858639
0.8889374726716223
0.9078543220710839


In [190]:
from sklearn.ensemble import AdaBoostClassifier

model1 = AdaBoostClassifier(random_state=1)
#model1.fit(eleicoes.loc[:,['recursos_de_pessoas_juridicas']], eleicoes.loc[:,['situacao_eleito']])
model1.fit(eleicoes.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']], eleicoes.loc[:,['situacao_eleito']])
#val_pred1=model1.predict(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas']])
val_pred1=model1.predict(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']])
#test_pred1=model1.predict(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas']])
test_pred1=model1.predict(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']])
val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)
print(model1.score(eleicoes.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes.loc[:,['situacao_eleito']]))
print(model1.score(eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes_val.loc[:,['situacao_eleito']]))
print(model1.score(eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','region_SE','region_NE','region_NO','region_SU','direction_R','direction_C']],eleicoes_val2.loc[:,['situacao_eleito']]))

C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9132853403141361
0.9077393965894185
0.912681000438789


In [198]:
from sklearn.neighbors import KNeighborsClassifier

model2 = KNeighborsClassifier()
model2.fit(eleicoes.loc[:,['ocupacao_DEPUTADO','recursos_proprios']], eleicoes.loc[:,['situacao_eleito']])
val_pred2=model2.predict(eleicoes_val.loc[:,['ocupacao_DEPUTADO','recursos_proprios']])
test_pred2=model2.predict(eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_proprios']])
val_pred2=pd.DataFrame(val_pred2)
test_pred2=pd.DataFrame(test_pred2)
model2.score(eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_proprios']],eleicoes_val2.loc[:,['situacao_eleito']])

C:\Users\tclem\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


0.863975427819219

In [197]:
from sklearn.ensemble import AdaBoostClassifier

model3 = AdaBoostClassifier(random_state=1)
model3.fit(eleicoes.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_fisicas','region_SU','region_SE','region_NE','region_NO','direction_R','direction_C']], eleicoes.loc[:,['situacao_eleito']])
val_pred3=model3.predict(eleicoes_val.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_fisicas','region_SU','region_SE','region_NE','region_NO','direction_R','direction_C']])
test_pred3=model3.predict(eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_fisicas','region_SU','region_SE','region_NE','region_NO','direction_R','direction_C']])
val_pred3=pd.DataFrame(val_pred3)
test_pred3=pd.DataFrame(test_pred3)
print(model3.score(eleicoes.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_fisicas','region_SU','region_SE','region_NE','region_NO','direction_R','direction_C']],eleicoes.loc[:,['situacao_eleito']]))
print(model3.score(eleicoes_val.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_fisicas','region_SU','region_SE','region_NE','region_NO','direction_R','direction_C']],eleicoes_val.loc[:,['situacao_eleito']]))
print(model3.score(eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_fisicas','region_SU','region_SE','region_NE','region_NO','direction_R','direction_C']],eleicoes_val2.loc[:,['situacao_eleito']]))

C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9136125654450262
0.9042413642326191
0.8968845985081176


In [217]:
from sklearn.linear_model import LogisticRegression

#df_val=pd.concat([eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], val_pred1,val_pred2],axis=1)
df_val=pd.concat([eleicoes_val.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], val_pred1,val_pred2,val_pred3],axis=1)
#df_test=pd.concat([eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], test_pred1,test_pred2],axis=1)
df_test=pd.concat([eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], test_pred1,test_pred2,test_pred3],axis=1)

model = LogisticRegression()
model.fit(df_val,eleicoes_val.loc[:,['situacao_eleito']])
print(model.score(df_val,eleicoes_val.loc[:,['situacao_eleito']]))
print(model.score(df_test,eleicoes_val2.loc[:,['situacao_eleito']]))

C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9164844774814167
0.9157525230364195


In [267]:
from sklearn.ensemble import RandomForestClassifier

#df_val=pd.concat([eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], val_pred1,val_pred2],axis=1)
df_val=pd.concat([eleicoes_val.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], val_pred1,val_pred2,val_pred3],axis=1)
#df_test=pd.concat([eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], test_pred1,test_pred2],axis=1)
df_test=pd.concat([eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], test_pred1,test_pred2,test_pred3],axis=1)

model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion="gini",
                               max_depth=15, max_features="auto", max_leaf_nodes=10,
                               min_samples_leaf=3,
                               min_samples_split=3, min_weight_fraction_leaf=0.0,
                               n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
                               verbose=0, warm_start=False)
model.fit(df_val,eleicoes_val.loc[:,['situacao_eleito']])
print(model.score(df_val,eleicoes_val.loc[:,['situacao_eleito']]))
print(model.score(df_test,eleicoes_val2.loc[:,['situacao_eleito']]))

C:\Users\tclem\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.929602098819414
0.9144361562088635


In [272]:



from sklearn.svm import SVC

#df_val=pd.concat([eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], val_pred1,val_pred2],axis=1)
df_val=pd.concat([eleicoes_val.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], val_pred1,val_pred2,val_pred3],axis=1)
#df_test=pd.concat([eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], test_pred1,test_pred2],axis=1)
df_test=pd.concat([eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], test_pred1,test_pred2,test_pred3],axis=1)

model = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
model.fit(df_val,eleicoes_val.loc[:,['situacao_eleito']])
print(model.score(df_val,eleicoes_val.loc[:,['situacao_eleito']]))
print(model.score(df_test,eleicoes_val2.loc[:,['situacao_eleito']]))

C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TypeError: must be real number, not str

In [219]:
from sklearn.ensemble import GradientBoostingClassifier

#df_val=pd.concat([eleicoes_val.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], val_pred1,val_pred2],axis=1)
df_val=pd.concat([eleicoes_val.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], val_pred1,val_pred2,val_pred3],axis=1)
#df_test=pd.concat([eleicoes_val2.loc[:,['recursos_de_pessoas_juridicas','recursos_proprios']], test_pred1,test_pred2],axis=1)
df_test=pd.concat([eleicoes_val2.loc[:,['ocupacao_DEPUTADO','recursos_de_pessoas_juridicas','recursos_de_pessoas_fisicas','direction_R','direction_C']], test_pred1,test_pred2,test_pred3],axis=1)

model = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(df_val,eleicoes_val.loc[:,['situacao_eleito']])
print(model.score(df_val,eleicoes_val.loc[:,['situacao_eleito']]))
print(model.score(df_test,eleicoes_val2.loc[:,['situacao_eleito']]))

C:\Users\tclem\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9182334936598163
0.9148749451513822


Perceba que dividimos os dados em dois seguimentos: as eleições de 2006 e 2010 foram separadas para validação e treinamento. Já a eleição de 2014 será nossa base de testes. Para o a validação, dividimos a base de treino aleatoriamente entre treino e validação (75% para treino e 25% para validação). Após a fase de validação, identificados os parâmetros mais adequados, retreinamos nosso modelo com a base de treinamento completa.

In [ ]:
#eleicoes_2014.head(10)
#eleicoes_2006 = eleicoes[eleicoes['ano'] == 2006]
#treino = eleicoes_2006.loc[:,[
#       'uf', 'quantidade_doacoes', 'total_receita', 'partido', 'quantidade_doadores', 
#       'recursos_de_outros_candidatos/comites',
#       'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
#       'recursos_proprios', 'quantidade_despesas', 'quantidade_fornecedores',
#       'total_despesa', 'cargo', 'sexo', 'grau',
#       'estado_civil', 'ocupacao', 'votos'
#    ]]

In [ ]:

#eleicoes.loc[eleicoes['ano'] == 2014].head(10)
#eleicoes_2010 = eleicoes[eleicoes['ano'] == 2010]
#validacao = eleicoes_2010.loc[:,[
#       'uf', 'quantidade_doacoes', 'total_receita', 'partido', 'quantidade_doadores', 
#       'recursos_de_outros_candidatos/comites',
#       'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
#       'recursos_proprios', 'quantidade_despesas', 'quantidade_fornecedores',
#       'total_despesa', 'cargo', 'sexo', 'grau',
#       'estado_civil', 'ocupacao', 'votos'
#    ]]

In [ ]:
#eleicoes_2006_a_2010 = eleicoes[eleicoes['ano'] == 2010 || eleicoes['ano'] == 2006]
#treino_final = eleicoes_2006_a_2010.loc[:,[
#       'uf', 'quantidade_doacoes', 'total_receita', 'partido', 'quantidade_doadores', 
#       'recursos_de_outros_candidatos/comites',
#       'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
#       'recursos_proprios', 'quantidade_despesas', 'quantidade_fornecedores',
#       'total_despesa', 'cargo', 'sexo', 'grau',
#       'estado_civil', 'ocupacao', 'votos'
#    ]]

In [ ]:
#eleicoes_2014 = eleicoes[eleicoes['ano'] == 2014]
#teste_final = eleicoes_2014.loc[:,[
#       'uf', 'quantidade_doacoes', 'total_receita', 'partido', 'quantidade_doadores', 
#       'recursos_de_outros_candidatos/comites',
#       'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
#       'recursos_proprios', 'quantidade_despesas', 'quantidade_fornecedores',
#       'total_despesa', 'cargo', 'sexo', 'grau',
#       'estado_civil', 'ocupacao', 'votos'
#    ]]

Vejamos agora as colunas de dados de treino e teste finais. Os atributos categóricos foram convertidos em numéricos utilizando uma estratégia de dumming.

In [ ]:
x_treino_final.head()

In [ ]:
y_treino_final.head()

In [ ]:
# Concatenando os preditores dos datasets de treino e teste
#all_data_treino = pd.concat((x_treino.loc[:,'uf':'ocupacao'],
#                      x_validacao.loc[:,'uf':'ocupacao']))

#all_data_final = pd.concat((x_treino_final.loc[:,'uf':'ocupacao'],
#                      x_teste_final.loc[:,'uf':'ocupacao']))
x_teste_final.head()

In [ ]:
y_teste_final.head()

## Pré-processamento

Aqui analisaremos os dados de forma mais ampla, fazendo modificações e preenchendo lacunas quando necessário.

In [ ]:
# Verificando quais transformação normaliza os dados
matplotlib.rcParams['figure.figsize'] = (12.0, 12.0)

votos = pd.DataFrame({"Votos":y_treino_final["votos"], "log(Votos)":np.log(y_treino_final["votos"]), \
                       "sqrt(Votos)":np.sqrt(y_treino_final["votos"]),"square(Votos)":np.square(y_treino_final["votos"])})
votos.hist()
plt.show()

Tomando a disbribução normal como a mais adequada para qualquer tratamento restatístico, faremos a transformação dos dados usando a função logarítmica. Tal função, como é possível observar nos gráficos acima, é a que mais aproxima o atributo `Votos` da distribuição normal.

Vale frizar que feremos esta transformação apenas em validação e treino, afim de identificar os melhores parâmetros. Na fase de testes usaremos a distribuição original para chegarmos a resultados mais inteligíveis. Em função disto, os RMSEs na validação e nos testes serão significativamente diferentes.

In [ ]:
#log transform the target:
y_treino = np.log1p(y_treino)#treino["votos"] = np.log1p(treino["votos"])
y_validacao = np.log1p(y_validacao)#validacao["votos"] = np.log1p(validacao["votos"])
####y_treino_final = np.log1p(y_treino_final)#treino_final["votos"] = np.log1p(treino_final["votos"])
####y_teste_final = np.log1p(y_teste_final)#teste_final["votos"] = np.log1p(teste_final["votos"])
#eleicoes["votos"] = np.log1p(eleicoes["votos"])

In [ ]:

#selecionando atributos numéricos
numeric_feats = eleicoes.dtypes[eleicoes.dtypes != "object"].index

#normalizando atributos numéricos exceto ano
x_treino[numeric_feats[1 :]] = np.log1p(x_treino[numeric_feats[1:]])
x_validacao[numeric_feats[1 :]] = np.log1p(x_validacao[numeric_feats[1:]])
####x_treino_final[numeric_feats[1 :]] = np.log1p(x_treino_final[numeric_feats[1:]])
####x_teste_final[numeric_feats[1 :]] = np.log1p(x_teste_final[numeric_feats[1:]])



#log transform skewed numeric features:
#numeric_feats = all_data_treino.dtypes[all_data_treino.dtypes != "object"].index

#skewed_feats = all_data_treino[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
##skewed_feats = x_validacao[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
#skewed_feats = skewed_feats[skewed_feats > 0.75]
#skewed_feats = skewed_feats.index

#all_data_treino[skewed_feats] = np.log1p(all_data_treino[skewed_feats])






#numeric_feats = all_data_final.dtypes[all_data_final.dtypes != "object"].index

#skewed_feats = all_data_final[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
##skewed_feats = x_teste_final[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
#skewed_feats = skewed_feats[skewed_feats > 0.75]
#skewed_feats = skewed_feats.index

#all_data_final[skewed_feats] = np.log1p(all_data_final[skewed_feats])

Note que os dados de teste e treino finais foram mantidos inalterados. Utilizaremos a função logarítmica apenas para encontrar os parâmetros de cana modelo (apenas na fase de validação).

In [ ]:
#filling NA's with the mean of the column:
#all_data_treino = all_data_treino.fillna(all_data_treino.mean())
#all_data_final = all_data_final.fillna(all_data_final.mean())
x_treino = x_treino.fillna(x_treino.mean())
x_validacao = x_validacao.fillna(x_validacao.mean())
x_treino_final = x_treino_final.fillna(x_treino_final.mean())
x_teste_final = x_teste_final.fillna(x_teste_final.mean())

y_treino = y_treino.fillna(y_treino.mean())
y_validacao = y_validacao.fillna(y_validacao.mean())
y_treino_final = y_treino_final.fillna(y_treino_final.mean())
y_teste_final = y_teste_final.fillna(y_teste_final.mean())


As lacunas vazias nos dados foram preenchidas com a média de cada coluna. Veja abaicho que já podemos observar os dummies para os atributos categóricos.

In [ ]:
x_teste_final.head(10)

#creating matrices for sklearn:
#X_treino = all_data_treino[:treino.shape[0]]
#X_validacao = all_data_treino[treino.shape[0]:]
#y_treino = treino.votos
#y_validacao = validacao.votos

#creating matrices for sklearn:
#X_treino_final = all_data_final[:treino_final.shape[0]]
#X_teste_final = all_data_final[treino_final.shape[0]:]
#y_treino_final = treino_final.votos
#y_teste_final = teste_final.votos

## Modelos

Descreveremos agora cada um de quatro modelos de predição distintos: regressão simples, com regularização Ridge, com regularização Lasso, K-NN, além de um modelo não visto em sala de aula chamado Random Forest.

In [ ]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, x_validacao, y_validacao.votos, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

def rmse_cv_final(model):
    rmse= np.sqrt(-cross_val_score(model, x_teste_final, y_teste_final.votos, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

### Sem regularização

In [ ]:
x_validacao.head(10)

In [ ]:
lm=LinearRegression()
lm.fit(x_treino,y_treino)
print("Intercept: " + str(lm.intercept_[0]))
print("RMSE: " + str(rmse_cv(lm).mean()))

coef_df = pd.DataFrame(lm.coef_[0],x_treino.columns,columns=['Coefficient'])
coef_df.head(10)

RMSE de mais de 22 mlhões de votos. Péssimo.

### Com regularização

#### Com regularização Ridge

In [ ]:

matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50]
cv_ridge = [rmse_cv(Ridge(alpha = alpha).fit(x_treino,y_treino)).mean() for alpha in alphas]
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation - Just Do It")
plt.xlabel("alpha")
plt.ylabel("rmse")

print("Mínmo RMSE: " + str(cv_ridge.min()))

RMSE de 0.968239, aceitável.

Seguiremos agora utilizando o melhor alpha para o modelo Ridge (alpha = 5), com os dados de treino completos e predizendo os dados de teste.

In [ ]:
modelo_ridge = Ridge(alpha = 5)
modelo_ridge.fit(x_treino_final,y_treino_final)
print("Intercept: " + str(modelo_ridge.intercept_[0]))
print("RMSE: " + str(rmse_cv_final(modelo_ridge).mean()))
coef_ridge = pd.DataFrame(modelo_ridge.coef_[0],x_treino.columns,columns=['Coefficient'])
coef_ridge.head(10)

Temos um RMSE real (de teste) de 33.981,52 votos. Perceba que, como já havíamos mencionado, o RMSE de teste (real) é bem diferentes do RMSE de validação. Isto ocorre pois realizamos a transformação logarítmica nas bases de validação e treino para validação, e não a fizemos nas bases de treino final e testes. Vale lembrar também que na prática calcular este RMSE de teste raramente é possível.

Comparamos agora, com base nos resíduos, a capacidade de predição do modelo já treinado para os próprios dados de treino e para os dados de validação

In [ ]:
#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":modelo_ridge.predict(x_treino_final)[:,0], "true":y_treino_final.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter", title = "Dados de treino")

#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":modelo_ridge.predict(x_teste_final)[:,0], "true":y_teste_final.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter",title = "Dados de teste")

O gráfico de resíduos parece um pouco enviezado, mas tanto validação quanto teste apresentam uma disperssão parecida, o que descarta overfitting.

#### Com regularização Lasso

In [ ]:

print("::::Validação:::::")

alphas = np.arange(0.0001,0.005,0.0001).tolist()

rmse_cv_lasso = [rmse_cv(Lasso(alpha = alpha, max_iter=8000).fit(x_treino,y_treino)).mean() 
            for alpha in alphas]

#print("RMSE validação: " + str(rmse_cv_lasso.mean()))
#print("Mínmo RMSE: " + str(rmse_cv_lasso.min()))

rmse_cv_lasso = pd.Series(rmse_cv_lasso, index = alphas)
rmse_cv_lasso.plot(title = "Validation - Just Do It")
plt.xlabel("alpha")
plt.ylabel("rmse")

rmse_cv_lasso.min()

#modelo_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005], max_iter=10000).fit(x_treino, y_treino.votos)
#rmse_cv_lasso = rmse_cv(modelo_lasso)
#rmse_lasso=rmse_cv_lasso.mean()
#print("Intercept validação: " + str(modelo_lasso.intercept_))
#print("RMSE validação: " + str(rmse_lasso))
#print("Mínmo RMSE: " + str(rmse_cv_lasso.min()))

#coef = pd.DataFrame(modelo_lasso.coef_, index = x_treino.columns, columns=['Coefficient']) #coeficientes
#coef.head(10)




RMSE de treino de 0.97145, maior que o da regressão com regularização Ridge, mas vamos proseguir mesmo assim.

In [ ]:


print("::::Teste:::::")

#alphas = np.arange(0.0001,0.005,0.0001).tolist()

modelo_lasso_final = Lasso(alpha = 0.0016, max_iter=8000).fit(x_treino_final,y_treino_final)
#rmse_cv_lasso_final = rmse_cv_final(Lasso(alpha = 0.0016, max_iter=8000))
rmse_cv_lasso_final = rmse_cv_final(modelo_lasso_final)

#modelo_lasso_final = LassoCV(alphas = [1, 0.1, 0.001, 0.0005], max_iter=10000).fit(x_treino_final, y_treino_final.votos)
#rmse_cv_lasso_final = rmse_cv_final(modelo_lasso_final)
rmse_lasso_final=rmse_cv_lasso_final.mean()
print("Intercept validação: " + str(modelo_lasso_final.intercept_))
print("RMSE validação: " + str(rmse_lasso_final))
#print("Mínmo RMSE: " + str(rmse_cv_lasso_final.min()))
coef = pd.DataFrame(modelo_lasso_final.coef_, index = x_treino_final.columns, columns=['Coefficient']) #coeficientes
coef.head(10)

RMSE de mais de 34.000 votos. Pior que o da regressão Ridge, como já prevíamos. A título de informação, veja abaixo quantos coeficientes foram removidos e mantidos durante a regressão Lasso (remover coeficientes pouco úteis é uma peculiaridade deste modelo)

In [ ]:
#coef = pd.Series(modelo_lasso.coef_, index = x_treino.columns)
coef_final = pd.Series(modelo_lasso_final.coef_, index = x_treino_final.columns)

#print("Train: Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

print("Final train: Lasso picked " + str(sum(coef_final != 0)) + " variables and eliminated the other " +  str(sum(coef_final == 0)) + " variables")


Podemos agora observar graficamente quais são os coeficientes são mais importantes:

In [ ]:
#imp_coef = pd.concat([coef.sort_values().head(10),
#                     coef.sort_values().tail(10)])

imp_coef_final = pd.concat([coef_final.sort_values().head(10),
                     coef_final.sort_values().tail(10)])

In [ ]:
#matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
#imp_coef.plot(kind = "barh")
#plt.title("Coefficients in the Lasso Model :::Treino:::")

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef_final.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model :::Teste:::")

Atributos dummies associados a um único candidado como `ocupacao_TÉCNICO EM EDIFICAÇÕES` ou `ocupacao_ATOR E DIRETOR DE ESPETÁCULOS PÚBLICOS` podem ganhar muita importância em função de sua votação particular. Os atributos que mais interferem no resultado são: `grau_de_instrucao`, `sigla_uf` e `ocupacao`, independente do processo de dumming. O atributo `sigla_uf`, de um modo geral parece ser bem relevante, indo de interferência mais negativa `sigla_uf_PR` para muito positiva `sigla_uf_CE`. Atributos aparentemente relevantes como `total_despesa` foram removidos. 

Veja abaixo que a distribuição de resíduos parece semelhante entre validação e este.

In [ ]:




#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

#preds = pd.DataFrame({"preds":modelo_lasso.predict(X_validacao), "true":y_validacao})
preds = pd.DataFrame({"preds":modelo_lasso_final.predict(x_treino_final), "true":y_treino_final.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title(":::Treino:::")

In [ ]:
x_treino_final.head(10)

In [ ]:



#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":modelo_lasso_final.predict(x_teste_final), "true":y_teste_final.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title(":::Teste:::")

O gráfico de resíduos parece oscilar em torno de 250.000 e -250.000 votos.

## Não paramétrica K-NN

Aqui trataremos da regressão ligada aos vizinhos mais próximos

In [ ]:



n_neighbors = [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90]
cv_knn = [rmse_cv(KNeighborsRegressor(n_neighbors = n_neighbor).fit(x_treino,y_treino)).mean() for n_neighbor in n_neighbors]
cv_knn = pd.Series(cv_knn, index = n_neighbors)
cv_knn.plot(title = "Validation - Just Do It")
plt.xlabel("neighbors")
plt.ylabel("rmse")
cv_knn.min()

#modelo_knn.predict(X_teste)

RMSE de 0.98, ruim, mas seguiremos em frente

In [ ]:
modelo_knn = KNeighborsRegressor(n_neighbors = 20)
modelo_knn.fit(x_treino,y_treino)

#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":modelo_knn.predict(x_validacao)[:,0], "true":y_validacao.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title(":::Treino:::")


#preds = pd.DataFrame({"preds":modelo_lasso_final.predict(x_teste_final), "true":y_teste_final.votos})
#preds["residuals"] = preds["true"] - preds["preds"]
#preds.plot(x = "preds", y = "residuals",kind = "scatter")
#plt.title(":::Teste:::")

In [ ]:
rmse_cv_final(modelo_knn).mean()

RMSE real de menos de 32.000 votos, o menor até agora, mesmo com um RMSE de validação não tão bom. Seguiremos para os resíduos.

In [ ]:
modelo_knn = KNeighborsRegressor(n_neighbors = 20)
modelo_knn.fit(x_treino_final,y_treino_final)

#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":modelo_knn.predict(x_teste_final)[:,0], "true":y_teste_final.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title(":::Teste:::")


#preds = pd.DataFrame({"preds":modelo_lasso_final.predict(x_teste_final), "true":y_teste_final.votos})
#preds["residuals"] = preds["true"] - preds["preds"]
#preds.plot(x = "preds", y = "residuals",kind = "scatter")
#plt.title(":::Teste:::")

Quanto aos resíduos, nada muito diferente.

## Random Forest

Trataremos aqui de Random Forest, único modelo deste relatório não tratado em sala de aula.

In [ ]:
n_estimators = [100, 400, 700, 900, 1000, 1200, 1400, 2000]
cv_rfr = [rmse_cv(RandomForestRegressor(max_depth=10, random_state=0, n_estimators = n_estimator).fit(x_treino,y_treino.votos)).mean() for n_estimator in n_estimators]
cv_rfr = pd.Series(cv_rfr, index = n_estimators)
cv_rfr.plot(title = "Validation - Just Do It")
plt.xlabel("estimators")
plt.ylabel("rmse")
cv_rfr.min()

#rfr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
#rfr.fit(X, y)
#print([cv.feature_importances_ for cv in cv_rfr])

RMSE de validação de 0.955, melhor até agora. Seguremos para os resíduos.

In [ ]:
rfr = RandomForestRegressor(max_depth=10, random_state=0, n_estimators=900)
rfr.fit(x_treino,y_treino)
#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":rfr.predict(x_validacao), "true":y_validacao.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

In [ ]:
rfr = RandomForestRegressor(max_depth=10, random_state=0, n_estimators=900)
rfr.fit(x_treino_final,y_treino_final)
#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":rfr.predict(x_teste_final), "true":y_teste_final.votos})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

Note acima uma distrubuição menos enviezada, porém ainda em um espectro muito similar ao dos demais modelos.

Veja abaixo que chegamos a um RMSE de 33.535. Com isto concluímos que o modelo K-NN, com RMSE de 31.941 prevê melhor os resultados para a eleição de 2014 com base em 2010 e 2006.

In [ ]:
rmse_cv_final(rfr).mean()

## Sumarização

Como pudemos observar, os melhores resultados são alcançados utilizando o modelo de regressão K-NN, tendo um RMSE no teste de aproximadamente 31.941 votos e apresentando uma disperssão no gráfico de resíduos menos concisa que a do modelo Random Forest, porém ambas enviezadas. É importante frizar que, embora a regressão K-NN seja superior em resultados, mesmo esta ainda apresenta um poder de previsão pouco útil na prática.

Como os resultados dos modelos foram bem próximos, é difícil dar um diagnóstico preciso a respeito do porquê um foi melhor que o outro. O úico modelo com desempenho que distoa dos demais é a regressão simples, pois não aplica nenhuma estratégia de ajuste e, com isto, leva grande desvantagem.